In [2]:
import sqlite3

## Posting lists

In [179]:
class PostingList(object):
    def __init__(self, coded_sequence=None):
        if coded_sequence is None:
            self.last_element = 0
            self.coded_sequence = bytearray()
        else:
            encoded_sequence = PostingList.decode_from_bytes_to_list(coded_sequence)
            self.last_element = encoded_sequence[-1]
            self.coded_sequence = coded_sequence

    @staticmethod
    def _to_128_coding(value):
        if value <= 0:
            raise ValueError("Value to be encoded must be positive")
        coded_bytes = []
        while value > 0:
            coded_bytes.append(value % 128)
            value //= 128
        return coded_bytes[::-1]
    
    @staticmethod 
    def _from_128_coding(bytes_list):
        encoded_value = 0
        for coded_byte in bytes_list:
            if coded_byte >= 128:
                raise ValueError(f"Coded byte cannot exceed value of 128, but was: {coded_byte}")
            encoded_value = encoded_value * 128 + coded_byte            
        return encoded_value
        
    @staticmethod
    def decode_from_bytes_to_list(coded_sequence):
        coded_value_bytes = []
        coded_value_increases = []
        for coded_byte in list(coded_sequence):
            coded_value_bytes.append(coded_byte % 128)
            if coded_byte >= 128:
                coded_value = PostingList._from_128_coding(coded_value_bytes)
                coded_value_increases.append(coded_value)
                coded_value_bytes = []
        coded_values = []
        for value_increase in coded_value_increases:
            last_value = coded_values[-1] if len(coded_values) > 0 else 0
            coded_values.append(last_value + value_increase)
        return coded_values

    def append(self, document_id):
        if document_id <= self.last_element:
            raise ValueError(f"Added document ID can't be less than the last document ID")
        value_increase = document_id - self.last_element
        coded_value_increase = PostingList._to_128_coding(value_increase)
        coded_value_increase[-1] += 128
        self.coded_sequence += bytearray(coded_value_increase)
        self.last_element = document_id
        
    def decode_to_list(self):
        return PostingList.decode_from_bytes_to_list(self.coded_sequence)

## Databse management

In [172]:
class IndexStorage(object):
    DATABASE_NAME = 'index.db'
    
    def __init__(self):
        self.connection = sqlite3.connect(IndexStorage.DATABASE_NAME)
        self.cursor = self.connection.cursor()
        self._create_tables_if_not_exists()
        
    def _create_tables_if_not_exists(self):
        self.cursor.execute("""
            CREATE TABLE IF NOT EXISTS indexed_terms
            (term TEXT PRIMARY KEY, posting_list BYTES)
        """)
        self.connection.commit()
        
    def add_indexed_term(self, term, posting_list):
        self.cursor.execute(
            "INSERT INTO indexed_terms (term, posting_list) VALUES (?, ?)",
            (term, posting_list.coded_sequence)
        )            
        self.connection.commit()


    def get_posting_list_of_term(self, term):
        return self.cursor.execute(
            "SELECT posting_list FROM indexed_terms WHERE term = (?) LIMIT 1",
            (term, )
        ).fetchone()[0]
        

In [173]:
x = IndexStorage()

In [174]:
term = "abc"
posting_list = PostingList()
posting_list.append(2)
posting_list.append(5)
posting_list.append(1000)

In [175]:
x.add_indexed_term(term, posting_list)

In [176]:
a = x.get_posting_list_of_term("abc")

In [181]:
aa = PostingList(a)

In [182]:
aa.decode_to_list()

[2, 5, 1000]

In [217]:
class OrderedList(object):
    def __init__(self, items):
        self.items = tuple(items)
        OrderedList._assert_ordered(self.items)

    @staticmethod
    def _assert_ordered(items):
        for i in range(1, len(items)):
            if items[i - 1] > items[i]:
                raise ValueError("Cannot initialize OrderedList with unsorted items")

    def __and__(self, other):
        items_intersection = []
        self_index, other_index = 0, 0
        while self_index < len(self.items) and other_index < len(other.items):
            if self.items[self_index] < other.items[other_index]:
                self_index += 1
            elif self.items[self_index] > other.items[other_index]:
                other_index += 1
            else:
                items_intersection.append(self.items[self_index])
                self_index += 1   
                other_index += 1            
        return OrderedList(items_intersection)

    def __or__(self, other):
        items_union = []
        self_index, other_index = 0, 0
        while self_index < len(self.items) or other_index < len(other.items):
            if other_index == len(other.items):
                items_union.append(self.items[self_index])                
                self_index += 1                
            elif self_index == len(self.items):
                items_union.append(other.items[other_index])                
                other_index += 1                              
            elif self.items[self_index] < other.items[other_index]:
                items_union.append(self.items[self_index])                
                self_index += 1
            elif self.items[self_index] > other.items[other_index]:
                items_union.append(other.items[other_index])                
                other_index += 1
            else:
                items_union.append(self.items[self_index])
                self_index += 1   
                other_index += 1
        return OrderedList(items_union)

In [218]:
x = OrderedList([2, 3, 5])

In [219]:
y = OrderedList([1, 2, 5, 8])

In [220]:
(x | y).items

(1, 2, 3, 5, 8)